In [157]:
import pandas as pd
import numpy as np

In [158]:
df = pd.read_csv("cleaned_data.csv")

In [159]:
#get rid of incomplete payments and deleted RSVPs
df = df[(df["RSVP'ed"] != "DELETED") & (df["Payment Status"] == "COMPLETE")]


In [160]:
#Create a refunds column
df["Refunded"] = False

# Iterate through the DataFrame to flag refunds and their corresponding original transactions
for i in range(1, len(df)):
    if df.loc[i]['Paid'] < 0:  # Check for refunds
        df.at[i, 'Refunded'] = True  # Mark refund row
        df.at[i - 1, 'Refunded'] = True  # Also mark the row above it (original transaction)

In [161]:
#Transform the dates to european format

df["Registration Date"] = pd.to_datetime(df["Registration Date"])  # Convert to datetime
df["Registration Date"] = df["Registration Date"].dt.strftime("%d/%m/%Y %H:%M:%S")  # Format it

C:\Users\valer\AppData\Local\Temp\ipykernel_34656\2843796707.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Registration Date"] = pd.to_datetime(df["Registration Date"])  # Convert to datetime


In [162]:
df["Payment Date"] = pd.to_datetime(df["Payment Date"])  # Convert to datetime
df["Payment Date"] = df["Payment Date"].dt.strftime("%d/%m/%Y %H:%M:%S")  # Format it

C:\Users\valer\AppData\Local\Temp\ipykernel_34656\1530286761.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Payment Date"] = pd.to_datetime(df["Payment Date"])  # Convert to datetime


In [163]:
df["Checked-In Date"] = pd.to_datetime(df["Checked-In Date"])  # Convert to datetime
df["Checked-In Date"] = df["Checked-In Date"].dt.strftime("%d/%m/%Y %H:%M:%S")  # Format it

C:\Users\valer\AppData\Local\Temp\ipykernel_34656\2366052488.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Checked-In Date"] = pd.to_datetime(df["Checked-In Date"])  # Convert to datetime


In [164]:
pd.unique(df["Registration Option"])

array(['Club Leaders', 'Gala Dinner Spring Sale Ticket - ENDED',
       'KIDS AREA TICKET', 'GAW early bird ticket - ENDED',
       'Class Reunion Committee Members', 'IE Faculty members',
       'GAW TICKET', 'EPIC Awards', 'GALA DINNER TICKET',
       'Special Invitation GAW 2024',
       'GAW + Gala Dinner Spring Sale Ticket - ENDED',
       'GAW Spring Sale Ticket - ENDED', 'Asociados AAA', 'GAR Team',
       'Special Invitation Gala 2024', 'Volunteers', 'GAR Team Gala',
       'Collaborators', 'GAW + GALA DINNER TICKET'], dtype=object)

In [165]:
#Change the names so they are more consistant
df["Registration Option"] = df["Registration Option"].replace({
    "Gala Dinner Spring Sale Ticket - ENDED":"Gala Dinner Spring Sale Ticket",
    "KIDS AREA TICKET":"Kids Area Ticket",
    "GAW early bird ticket - ENDED":"GAW Early Bird Ticket",
    "GAW TICKET":"GAW Ticket",
    "GALA DINNER TICKET":"Gala Dinner Ticket",
    "GAW + Gala Dinner Spring Sale Ticket - ENDED":"GAW + Gala Dinner Spring Sale Ticket",
    "GAW Spring Sale Ticket - ENDED":"GAW Spring Sale Ticket",
    "GAW + GALA DINNER TICKET":"GAW + Gala Dinner Ticket"
})

In [166]:
#Classify the tickets

gaw_tickets = ['Club Leaders','GAW Early Bird Ticket',
    'Class Reunion Committee Members', 'IE Faculty members',
    'GAW Ticket', 'EPIC Awards',
    'Special Invitation GAW 2024',
    'GAW + Gala Dinner Spring Sale Ticket', 'GAW Spring Sale Ticket',
    'Asociados AAA', 'GAR Team',
    'GAW + Gala Dinner Ticket']

gala_tickets = ['Gala Dinner Spring Sale Ticket',
    'EPIC Awards', 'Gala Dinner Ticket',
    'GAW + Gala Dinner Spring Sale Ticket',
    'Special Invitation Gala 2024',
    'GAR Team Gala','GAW + Gala Dinner Ticket']

kid_tickets = ["Kids Area Ticket"]

other_tickets = ['Volunteers', 'Collaborators']

In [167]:
# Create new columns using the lists
df["GAW"] = df["Registration Option"].isin([x for x in gaw_tickets]).astype(int)
df["GALA"] = df["Registration Option"].isin([x for x in gala_tickets]).astype(int)
df["KID"] = df["Registration Option"].isin([x for x in kid_tickets]).astype(int)
df["OTHER"] = df["Registration Option"].isin([x for x in other_tickets]).astype(int)

In [168]:
# Drop irrelevant columns
df = df.drop(
    [
    "Section (please, type N/A if this is not applicable to your program)",
    "Attendee's Comment", "Officer's Notes", "Quantity", "Unnamed: 40"
    ],
    axis="columns"
)

In [169]:
#Change type of paid to int as there's no decimals
df["Paid"] = df["Paid"].astype("int64")

In [170]:
#Categorize countries into regions

region_mapping = {
    "North America": ["Canada", "United States Of America", "Mexico", "Puerto Rico"],
    "South America": ["Brazil", "Argentina", "Colombia", "Chile", "Peru", "Ecuador", "Uruguay", "Venezuela", "Bolivia"],
    "Europe": ["Spain", "United Kingdom", "Germany", "France", "Switzerland", "Ireland", "Italy", "Portugal",
            "Netherlands", "Belgium", "Luxembourg", "Austria", "Sweden", "Finland", "Denmark", "Norway", 
            "Poland", "Czech Republic", "Hungary", "Lithuania", "Latvia", "Romania", "Bulgaria", "Russia",
            "Belarus", "Ukraine", "Monaco", "Georgia", "Croatia", "Greece"],
    "Middle East": ["United Arab Emirates", "Saudi Arabia", "Qatar", "Kuwait", "Bahrain", "Lebanon", "Israel", "Afganistan"],
    "Africa": ["South Africa", "Nigeria", "Algeria", "Egypt", "Ghana", "Namibia", "Liberia", "Kenya", "Burkina Faso"],
    "Asia & Oceania": ["India", "China", "Japan", "South Korea", "Bangladesh", "Kazakhstan", "Azerbaijan", "Singapore", 
            "Thailand", "Philippines", "Hong Kong", "Turkey", "Afghanistan", "Kyrgyztan", "Australia"],
    "Central America & Caribbean": ["Costa Rica", "Jamaica", "Dominican Republic"],
}

In [171]:
#Classifying function 

def get_region(country):
    for region, countries in region_mapping.items():
        if country in countries:
            return region
    return "Other"

In [172]:
#Apply function to create new column

df["Region"] = df["Country"].apply(lambda x: get_region(str(x)) if pd.notna(x) else "Unknown")

In [173]:
#Put region next to country
col_order = df.columns.tolist()
col_order.insert(col_order.index("Country") + 1, col_order.pop(col_order.index("Region")))
df = df[col_order]

In [174]:
df.columns

Index(['Account Type', 'Member Tags', 'User Tags', 'RSVP'ed',
       'Registration Date', 'Registration Option', 'Are you an IE Alumni?',
       'Year of graduation', 'Degree', 'Country', 'Region',
       'I'm interested in networking around ... (please choose one)',
       'Please let us know if you have any allergies or dietary restrictions:',
       'Which roles are you most interested in? (Mark all that apply)', 'Paid',
       'Payment Status', 'Payment Date', 'Checked-In Date', 'Net ID',
       'Gateway Number', 'Event Registration UID', 'Refunded', 'GAW', 'GALA',
       'KID', 'OTHER'],
      dtype='object')

In [175]:
#Rename columns
column_rename_map = {
    'Account Type': 'account_type',
    'Member Tags': 'member_tags',
    'User Tags': 'user_tags',
    "RSVP'ed": 'rsvp',
    'Registration Date': 'registration_date',
    'Registration Option': 'registration_option',
    'Are you an IE Alumni?': 'ie_alumni',
    'Year of graduation': 'graduation_year',
    'Degree': 'degree',
    'Country': 'country',
    'Region': 'region',
    "I'm interested in networking around ... (please choose one)": 'networking_preference',
    'Please let us know if you have any allergies or dietary restrictions:': 'allergies_dietary_restrictions',
    'Which roles are you most interested in? (Mark all that apply)': 'volunteer_roles',
    'Paid': 'paid',
    'Payment Status': 'payment_status',
    'Payment Date': 'payment_date',
    'Checked-In Date': 'checked_in_date',
    'Net ID': 'net_id',
    'Gateway Number': 'gateway_number',
    'Event Registration UID': 'event_registration_uid',
    'Refunded': 'refunded',
    'GAW': 'gaw',
    'GALA': 'gala',
    'KID': 'kid',
    'OTHER': 'other'
}

In [176]:
df.rename(columns=column_rename_map, inplace=True)

In [177]:
#extract tags to identify if its a class reunion
import re

In [178]:
def extract_study_programs(user_tags):
    if pd.isna(user_tags):  # Handle missing values
        return None
    
    # Updated regex pattern to match both full and short study program formats
    pattern = r'\b[A-Z]+(?:-[A-Za-z]+)?(?: -)? [A-Z]{3}-\d{4}\b'
    
    # Find all matching study-related tags
    matches = re.findall(pattern, user_tags)
    
    return matches if matches else None  # Return list of matches or None if no matches found

# Apply function to extract all study program tags
df['study_programs'] = df['user_tags'].apply(extract_study_programs)

In [179]:
# Expand the Study Programs column into multiple new columns
max_tags = df['study_programs'].dropna().apply(len).max()  # Find max number of tags per person

# Create new columns dynamically
for i in range(max_tags):
    df[f'study_programs {i+1}'] = df['study_programs'].apply(lambda x: x[i] if x and len(x) > i else None)

# Drop the original list column (optional)
df.drop(columns=['study_programs'], inplace=True)

In [180]:
# Define reunion tags with full years instead of two-digit years

reunion_tags = [
    "imba ene-2019", "mim feb-2019", "gxmba may-2018", "gxmba oct-2018", "imba sep-2018", "mim sep-2018", "mrcb oct-2013",
    "exmplst abr-2013", "imba abr-2013", "gxmba jun-2013", "gxmba nov-2013", "imba nov-2013", "mim sep-2013", "emba-en abr-2013",
    "emba-en nov-2008", "mim oct-2008", "imba abr-2008", "ixmba jun-2008", "gxmba may-2008", "imba nov-2008",
    "emba-es-s (exmba) oct-2008", "ixmba nov-2003", "mba feb-2004", "ixmba jun-2003", "imba-eng nov-2003", "emba-es-s feb-2003",
    "emba-es-s oct-2003", "emba-es-s feb-2004", "mba oct-2003", "emba-es-s (exmba) feb-1999", "mba feb-1999",
    "emba-es-s (exmba) oct-1998", "mba oct-1998", "mide oct-1998", "mide oct-1993", "mba oct-1993", "mba feb-1994",
    "mide feb-1989", "mba oct-1988", "bba sep-2015", "bbabir sep-2014", "bir sep-2015", "llbbir sep-2014", "bar sep-2014",
    "mbd abr-2018", "mbds oct-2018", "mbd-pt ene-2018", "mcs oct-2018", "mamd feb-2018", "mir oct-2018", "mid oct-2018",
    "mamd feb-2013", "mbd oct-2013", "mir oct-2013", "mir oct-2008", "llmibl oct-2018", "maj feb-2014", "maj oct-2013",
    "majd oct-2013", "maf oct-2013", "maf oct-2008", "maj-pwc sep-2008", "maj oct-2008", "majp nov-2008", "maj feb-2009",
    "maj oct-2003", "maj feb-2004", "maf feb-1999", "maf oct-1998", "maj feb-1999", "maj oct-1998", "maj oct-1993",
    "maj feb-1994", "maf oct-1993", "maf feb-1994", "maf oct-1988", "maj oct-1988", "maj feb-1989", "mbap oct-2003",
    "ie-smu mba nov-2018", "miaf ene-2014", "gmba-esp-s nov-2012", "emba-es-s (exmbaq) may-2008", "emba-es-s (exmba) oct-1993",
    "gmba-en-bl oct-2017", "mcc oct-2013", "gmba-en-bf sep-2013", "emba-es-s feb-2014", "emba-en nov-2012", "exmplst abr-2018",
    "gmba-en-bf nov-2012", "emba-es-s oct-2013", "mba 1984", "bba sep-2010", "mim feb-2014"
]


In [181]:
# Define the reunion years
reunion_years = {2019, 2014, 2009, 2004, 1999, 1994, 1989, 1984, 1979, 1974, 1969}

In [182]:
# Find all columns that contain study program data
study_program_columns = [col for col in df.columns if "study_program" in col]

In [183]:
def check_reunion(row):
    # Get all study programs for a person, ignoring NaNs
    study_programs = row[study_program_columns].dropna().tolist()
    
    # Normalize to lowercase
    study_programs = [program.lower() for program in study_programs]
    
    # Check if any study program matches a reunion tag
    if any(program in reunion_tags for program in study_programs):
        return "Yes"
    
    # If not in reunion tags, check the graduation year logic
    try:
        year = int(row["graduation_year"])
        if year in reunion_years:
            return "Possible"
        else:
            return "No"
    except (ValueError, TypeError):
        return "Unknown"

In [184]:
# Apply function to determine if someone is celebrating a class reunion
df["class_reunion"] = df.apply(check_reunion, axis=1)

In [185]:
# Ensure partners also get the same class reunion status
def assign_partner_reunion(df):
    alumni_reunion_status = {}
    
    for _, row in df.iterrows():
        if row["ie_alumni"] == "Yes" and row["class_reunion"] in ["Yes", "Possible"]:
            key = (row["user_tags"], row["net_id"])
            alumni_reunion_status[key] = row["class_reunion"]
    
    for idx, row in df.iterrows():
        if row["ie_alumni"] == "No":
            key = (row["user_tags"], row["net_id"])
            if key in alumni_reunion_status:
                df.at[idx, "class_reunion"] = alumni_reunion_status[key]

assign_partner_reunion(df)

In [186]:
df.columns

Index(['account_type', 'member_tags', 'user_tags', 'rsvp', 'registration_date',
       'registration_option', 'ie_alumni', 'graduation_year', 'degree',
       'country', 'region', 'networking_preference',
       'allergies_dietary_restrictions', 'volunteer_roles', 'paid',
       'payment_status', 'payment_date', 'checked_in_date', 'net_id',
       'gateway_number', 'event_registration_uid', 'refunded', 'gaw', 'gala',
       'kid', 'other', 'study_programs 1', 'study_programs 2',
       'study_programs 3', 'study_programs 4', 'study_programs 5',
       'study_programs 6', 'study_programs 7', 'study_programs 8',
       'study_programs 9', 'study_programs 10', 'study_programs 11',
       'class_reunion'],
      dtype='object')

In [187]:
#We classify the alumni per school

# Define mapping of study programs to schools
program_to_school = {
    "IMBA": "Business School",
    "MBA": "Business School",
    "MIM": "Business School",
    "GXMBA": "Business School",
    "EMBA": "Business School",
    "IXMBA": "Business School",
    "GMBA": "Business School",
    "LLMIBL": "Law School",
    "MAJ": "Law School",
    "MAJD": "Law School",
    "MAF": "Law School",
    "BAR": "School of Architecture and Design",
    "BBA": "Business School",
    "BBABIR": "School of Social Sciences",
    "BIR": "School of Social Sciences",
    "LLBBIR": "School of Social Sciences",
    "LLB": "Law School",
    "MBD": "School of Science and Technology",
    "MAMD": "School of Architecture and Design",
    "MIR": "School of Social Sciences",
    "MID": "School of Architecture and Design",
    "MIDE": "Business School",
    "IEBEXMBA": "Business School",
    "MDGM": "Business School",
    "MRCB": "Business School",
    "MIF": "Business School",
    "TechMBA": "School of Science and Technology",
    "MIM-English": "Business School",
    "MIM-Spanish": "Business School",
    "GMRED": "Business School",
    "EMBA-EN-BL": "Business School",
    "EXMBA": "Business School",
    "MVDM": "Business School",
    "MAJ-PWC": "Law School",
    "EXMPLT": "Business School",
    "MCC": "Business School",
    "Master in Marketing": "Business School",
    "GOMBA": "Business School",
    "Communication & Digital media": "Business School",
    "Master in Costumer Behaviour": "Business School",
    "Master in Marketing": "Business School",
    "Law Studies": "Law School",
    "International Relations": "School of Social Sciences",
    "Coaching & Leadership": "Business School",
    "Architecture Studies": "School of Architecture and Design",
    "EXEC.ED": "Executive Education",
    "PHD": "Business School",
    "Other": "Executive Education",
    "Unknow": "Executive Education"
}

In [188]:
# Standardized degree categories
degree_mapping = {
    # imba - international mba
    'IMBA': [
        'imba', 'international mba', 'international mba (english)', 'international mba (español)',
        'imba ', 'imba 2013', 'imba 2012', 'imba + master in business analytics and big data',
        'international mba ', 'imba (mba)', 'international mba mide', 'international mba (mide)', 'mide'
    ],

    # gomba - global online mba
    'GOMBA': [
        'gomba', 'global online mba', 'global mba english blended', 'global mba+ español semanal',
        'global mba english blended  (face to face every six weeks)', 'global mba+ español semanal'
    ],

    # emba - executive mba
    'EMBA': [
        'emba', 'executive mba', 'executive mba english', 'executive mba español', 'executive mba blended',
        'executive mba english blended', 'executive mba español blended', 'executive mba español presencial semanal',
        'executive mba+ english bi-weekly', 'emba2022', 'executive master of business administration',
        'executive mba ', 'executive mba español blended online', 'executive mba español blended',
        'executive mba español blended online', 'executive mba part-time', 'executive mba blended online',
        'executive mba+ english bi-weekly', 'exmba 90', 'exmba', 'executive mba bi-weekly',
        'executive mba español blended', 'executive mba español blended online', 'executive mba part time'
    ],

    # gxmba - global executive mba
    'GXMBA': [
        'gxmba', 'global executive mba', 'international executive mba', 'ie brown executive mba',
        'global executive mba bi-weekly', 'international executive mba bi-weekly'
    ],

    # standard mba (if it doesn't fit into the categories above)
    'MBA': [
        'mba', 'mba part-time', 'mba ', 'mba - iim lucknow', 'mba (mba)', 'mba part time',
        'mbap', 'global mba', 'global mba ', 'global mba english', 'global mba blended', 
        'mba international', 'mba global', 'global m.b.a.', 'mba - iim lucknow', 'master in business administration',
        'master business administration'
    ],
    
    # mim - master in management
    'MIM': [
        'master in management', 'mim', 'mmim', 'masters in management', 'master in international management'
    ],
    
    # mif - master in finance
    'MIF': [
        'master in finance', 'master in advanced finance', 'master in finance & master in management'
    ],
    
    # master in marketing
    'Master in Marketing': [
        'master in marketing', 'master in marketing management', 'master in digital marketing',
        'master in digital marketing part time', "mmm"
    ],
    
    # mbabd - master in business analytics & big data
    'MBD': [
        'master in business analytics and big data', 'business analytics', 'mbd pt 2022'
    ],

    'Master in Costumer Behaviour': [
        'master in market research &amp; consumer behavior', 'master in market research & consumer behavior',
        'master in customer experience and innovation' 
    ],    
    
    # bba - bachelor in business administration
    'BBA': [
        'bba', 'bachelor in business administration', 'bachelor in business administration madrid campus',
        'bachelor in business administration segovia campus', 'dual degree bachelor in business administration', 
        'grado en administración de empresas campus de segovia', 'grado en administración de empresas campus de madrid'
    ],
    
    # law masters
    'Law Studies': [
        'llm', 'law', 'master in laws', 'doble máster en abogacía', 'doble máster en abogacía y asesoría jurídica',
        'llm in international business law', 'master en asesoría jurídica de empresas', 'master in legaltech',
        'master in legal tech', 'doble master en abogacía, emprendimiento y tecnología', 'maj 1999', "maj", 'máster ejecutivo en asesoría jurídica de los negocios',
        'master en asesoría jurídica/fiscal ie/pwc tax and legal services'
    ],
    
    # phd programs
    'PHD': [
        'phd', 'ph. d.', 'ph.d', 'ph.d. candidate', 'phd in medicine'
    ],
    
    # architecture
    'Architecture Studies': [
        'bachelor in architecture', 'architecture', 'mamd - master in architectural management and design',
        'master in architectural management and design', 'bachelor in architectural studies'
    ],
    
    # international relations & politics
    'International Relations': [
        'master in international relations', 'ms. international relations', 'international relations', 'master in international development'
    ],
    
    # communication & digital media
    'Communication & Digital media': [
        'master in corporate communication', 'master in corporate and marketing communication',
        'bachelor in communication and digital media segovia campus', 'mdcm', 'master in visual and digital media',
        'bachelor in design', 'master in graphic design'
    ],
    
    # coaching & leadership
    'Coaching & Leadership': [
        'psychology', 'executive coaching', 'executive coaching ', 'programa de dirección en coaching ejecutivo',
        'executive master in positive leadership & transformation', 'master in positive leadership and transformation',
        'executive master in positive leadership and strategy', 'programa de dirección estratégica de personas y talento'
    ],
    
    "GMRED": ["gmred", "global master in real estate development" ],
    
    # other degrees
    'EXEC.ED': [
        'advanced management program', 'amp intensive', 'global senior management program', 'gsmp',
        'senior management program', 'owners scale-up program blended', 'energy forum', 'exmba 90',
        'advanced management program zaragoza', 'advanced management program madrid', 'programa ejecutivo', "advanced master program",
        "amp 2013"
    ]
}


In [189]:
# Function to clean and standardize degrees
def clean_degree(degree, alumni_status):
    if pd.isna(degree) or str(degree).strip().lower() in ['na', 'n/a', '-', 'not applicable']:
        return "Unknown"
    
    degree_lower = str(degree).strip().lower()
    
    # Assign categories based on predefined mapping
    for standard, variations in degree_mapping.items():
        if any(variation in degree_lower for variation in variations):
            return standard
    
    # Assign "Not Applicable" for non-alumni
    if alumni_status.lower() == "no":
        return "Not Applicable"
    
    return "Other"

In [190]:
# Apply function to clean degrees
df['degree_cleaned'] = df.apply(lambda x: clean_degree(x['degree'], x['ie_alumni']), axis=1)

In [191]:
# Function to determine school based on both study programs and cleaned degree
def get_school(row):
    study_programs = row[study_program_columns].dropna().tolist()  # Get all study programs

    # Check study program mappings first
    for program in study_programs:
        for key in program_to_school:
            if key in program:
                return program_to_school[key]  # Return first match

    # If no match, check cleaned degree
    cleaned_degree = row["degree_cleaned"]
    if cleaned_degree in program_to_school:
        return program_to_school[cleaned_degree]

    return "Unknown"  # If no match is found, label as unknown

# Apply function to determine the school
df["school"] = df.apply(get_school, axis=1)

In [192]:
df["ie_alumni"] = df["ie_alumni"].fillna("").replace("", "No")

In [193]:
df.loc[df["ie_alumni"] == "No", "school"] = "Not Applicable"

In [194]:
df.loc[df["ie_alumni"] == "No", "graduation_year"] = 0

In [195]:
df["graduation_year"] = pd.to_numeric(df["graduation_year"], errors='coerce')

In [196]:
df["graduation_year"] = df["graduation_year"].convert_dtypes()

In [197]:
df["registration_option"]

0                         Club Leaders
1       Gala Dinner Spring Sale Ticket
2                         Club Leaders
3                         Club Leaders
4       Gala Dinner Spring Sale Ticket
                     ...              
1152                        GAW Ticket
1153                        GAW Ticket
1154                        GAW Ticket
1155                        Volunteers
1156                        Volunteers
Name: registration_option, Length: 1157, dtype: object

In [198]:
def classify_ticket_buyer(option):
    if pd.isna(option):
        return "Unknown"
    if "club leaders" in option.lower():
        return "Club Leader"
    elif "class reunion committee members" in option.lower():
        return "Class Reunion Committe Member"
    else:
        return "Normal Attendee"

In [199]:
df["ticket_buyer"] = df["registration_option"].apply(classify_ticket_buyer)

In [200]:
def bundle_ticket(option):
    if pd.isna(option):
        return "Unknown"
    if "gaw + gala dinner" in option.lower():
        return "Bundle Ticket"
    else:
        return "Normal Ticket"

In [201]:
df["bundle_ticket"] = df["registration_option"].apply(bundle_ticket)

In [202]:
# Define the desired column order
base_columns = ['account_type', 'rsvp', 'registration_date',
    'registration_option', 'ie_alumni', 'graduation_year', 'degree','class_reunion', 'school',
    'country', 'region', 'networking_preference',
    'allergies_dietary_restrictions', 'volunteer_roles', 'paid',
    'payment_status', 'payment_date', 'checked_in_date', 'net_id',
    'gateway_number', 'event_registration_uid', 'refunded', 'gaw', 'gala',
    'kid', 'other','member_tags', 'user_tags', "school", "degree_cleaned", "ticket_buyer", "bundle_ticket"
]

In [203]:
study_program_columns = sorted(
    [col for col in df.columns if col.startswith("study_programs")],
    key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else 0
)

# Combine static columns with dynamically detected "Study Program" columns
final_column_order = base_columns + study_program_columns

# Reorder the dataframe, ignoring any missing columns
df_reordered = df[[col for col in final_column_order if col in df.columns]]

In [204]:
df_sorted = df_reordered.sort_values(by="payment_date", ascending=True)

In [205]:
# Define ticket categories
ticket_categories = {
    "Early Bird": [
        "GAW Early Bird Ticket"
    ],
    "Spring Sale": [
        "Gala Dinner Spring Sale Ticket", 
        "GAW Spring Sale Ticket", 
        "GAW + Gala Dinner Spring Sale Ticket"
    ],
    "Full Price": [
        "GAW Ticket", 
        "Gala Dinner Ticket", 
        "GAW + Gala Dinner Ticket", 
        "Kids Area Ticket"
    ],
    "Special Price": [
        "Class Reunion Committee Members", 
        "Club Leaders", 
        "Asociados AAA"
    ],
    "Free": [
        "IE Faculty members", 
        "Volunteers", 
        "GAR Team", 
        "Special Invitation GAW 2024", 
        "EPIC Awards", 
        "GAR Team Gala", 
        "Special Invitation Gala 2024", 
        "Collaborators"
    ]
}

In [206]:
# Create a mapping dictionary
category_mapping = {option: category for category, options in ticket_categories.items() for option in options}

# Apply the mapping to create a new column
df_sorted["ticket_category"] = df_sorted["registration_option"].map(category_mapping)

In [207]:
df_sorted.to_excel("ready_data.xlsx", index=False)